# Final Multi-Class Classifictaion

The Mulit-Class Classification with the pretrained model Bert is the best overall with it's accuracy and robustness. It exceeded my expectations and in this model I will not split my data but rather use 100% of my tiny dataset for the maximum accuracy on actual data.

## The Model
This model is considered the final model for this project as I have not splitted any test data, just validation in order to get the most out of my tiny dataset for my chatbot.

This model can be found here:

https://huggingface.co/distilbert-base-uncased?text=The+goal+of+life+is+%5BMASK%5D.

I heavily utilized a guide in order to achieve this in Tensorflow and not PyTorch as that is what I wanted to use for this project. 

This guide can be found here:

https://www.sunnyville.ai/fine-tuning-distilbert-multi-class-text-classification-using-transformers-and-tensorflow/



## Prepping Our Libraries/Classes

In [2]:
# Installing the latest version of transformers
!pip install git+https://github.com/huggingface/transformers.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-mjxoto3l
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-mjxoto3l
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.6 MB 5.1 MB/s 
     |████████████████████████████████| 163 kB 45.7 MB/s 
  Created wheel for transformers: filename=transformers-4.25.0.dev0-py3-none-any.whl size=5645784 sha256=3edad3f395c4c3db4b8159f73090667b3ca322d4e4c20dd68abb100112c6e7dd
  Stored in directory: /tmp/pip-ephem-wheel-cache-f3t3pi0e/wheels/90/a5/44/6bcd83827c8a60628c5ad602f429cd5076bcce5f2a90054947
Successfully built transformers


In [3]:
# Importing our needed libraries
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf
import pandas as pd

## Data Processing
Here we will load in my inputs and category dataset from github.

This Dataset can be found here:

https://github.com/BridgetteBXP13/CS-4395.001---Human-Language-Technologies/blob/main/Chatbot/Data/Inputs.csv

In [4]:
# Import our data
url = 'https://raw.githubusercontent.com/BridgetteBXP13/CS-4395.001---Human-Language-Technologies/main/Chatbot/Data/Inputs.csv'
df = pd.read_csv(url)
print("\nOur loaded dataframe:\n")
df.head


Our loaded dataframe:



<bound method NDFrame.head of                                                  Input        Category
0                                Snakes are aggressive        Behavior
1                                 Are they aggressive         Behavior
2    Snakes will not bite unless you try to approac...        Behavior
3                              Do snakes like to bite?        Behavior
4                                  Snakes chase people        Behavior
..                                                 ...             ...
325                             Snakes are emotionless           Brain
326                               Can snakes grow hair            Body
327                                    Snakes are mean        Behavior
328                            Why should snakes exist  Snake Benefits
329                                Are snakes any good  Snake Benefits

[330 rows x 2 columns]>

In [5]:
# Display our categories and their counts
print("\nNumber of Categories:", len(df.Category.unique()))
print("\nOur Categories:")
print(df.Category.unique())
print("\nCounts of each Category:")
print(df.Category.value_counts())


Number of Categories: 51

Our Categories:
['Behavior' 'Body' 'Brain' 'Bye' 'Creators' 'Crossbreeding'
 'Dangerous Snakes' 'Deaf' 'Definition' 'Diamond' 'Dislocate Jaws' 'Eat'
 'Endangered' 'Escape' 'Evil' 'Eyesight' 'Fear' 'Flying snakes' 'Generic'
 'Greeting' 'Infared' 'Kill Snakes' 'Lay Eggs' 'Legless Lizard' 'Legs'
 'Live Forever' 'Lizards Discourage' 'Misunderstand' 'Mother Snake'
 'Music' 'Musk' 'Name' 'Pairs' 'Pet Snakes' 'Poisonous' 'Pupils' 'Purpose'
 'Rattle' 'Scared' 'Size' 'Slimy' 'Smell' 'Snake Attraction'
 'Snake Benefits' 'Snake Bite' 'Suffication' 'Swim' 'Tails' 'Topics'
 'Understand' 'Venomous']

Counts of each Category:
Pet Snakes            17
Body                  16
Behavior              12
Eat                   11
Understand            10
Eyesight              10
Venomous              10
Pairs                  8
Brain                  8
Misunderstand          8
Generic                8
Legs                   8
Snake Benefits         8
Kill Snakes            7
Disl

In [6]:
# Adding a column of category codes (cat codes)
df['encoded_cat'] = df.Category.astype('category').cat.codes
print("\nOur new dataframe with cat codes:\n")
df.head()


Our new dataframe with cat codes:



,Input,Category,encoded_cat
0,Snakes are aggressive,Behavior,0
1,Are they aggressive,Behavior,0
2,Snakes will not bite unless you try to approac...,Behavior,0
3,Do snakes like to bite?,Behavior,0
4,Snakes chase people,Behavior,0


In [7]:
# Converting Our Data to Lists
data_input_texts = df.Input.to_list()
data_cats = df.encoded_cat.to_list()
print("\nThe first five of our input text list:")
print(data_input_texts[:5])
print("\nThe first five cat codes for the input text list:")
print(data_cats[:5])


The first five of our input text list:
['Snakes are aggressive', 'Are they aggressive ', 'Snakes will not bite unless you try to approach/handle them.', 'Do snakes like to bite?', 'Snakes chase people']

The first five cat codes for the input text list:
[0, 0, 0, 0, 0]


In [8]:
# Splitting Into Train, Test, & Validation
from sklearn.model_selection import train_test_split

# Split Train & Validate
train_inputs, val_inputs, train_cats, val_cats = train_test_split(data_input_texts, data_cats, test_size=0.1, random_state=1234)
# Split Train & Test
#train_inputs, test_inputs, train_cats, test_cats = train_test_split(train_inputs, train_cats, test_size=.1, random_state=1234)

## Tensorflow

### Creating Model and Prepping Data

In [9]:
# Tokenizing and Encoding our input text
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_inputs, truncation=True, padding=True)
val_encodings = tokenizer(val_inputs, truncation=True, padding=True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [10]:
# Creating our Tensorflow Dataset Objects
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_cats
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_cats
))

### Fine Tuning the Model

In [11]:
# Building & Compliling the Model
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=51)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_transform', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [12]:
# Training & Fine Tuning the Model
model.fit(train_dataset.shuffle(1000).batch(32), epochs=100, batch_size=32,
          validation_data=val_dataset.shuffle(1000).batch(32))

Epoch 1/100
10/10 [==============================] - 67s 5s/step - loss: 3.9089 - accuracy: 0.0471 - val_loss: 3.9004 - val_accuracy: 0.0303
Epoch 2/100
10/10 [==============================] - 50s 5s/step - loss: 3.8024 - accuracy: 0.0640 - val_loss: 3.8461 - val_accuracy: 0.0303
Epoch 3/100
10/10 [==============================] - 49s 5s/step - loss: 3.6790 - accuracy: 0.1044 - val_loss: 3.7333 - val_accuracy: 0.0303
Epoch 4/100
10/10 [==============================] - 50s 5s/step - loss: 3.5068 - accuracy: 0.1886 - val_loss: 3.6225 - val_accuracy: 0.0909
Epoch 5/100
10/10 [==============================] - 49s 5s/step - loss: 3.3109 - accuracy: 0.3737 - val_loss: 3.4848 - val_accuracy: 0.2727
Epoch 6/100
10/10 [==============================] - 51s 5s/step - loss: 3.0514 - accuracy: 0.5421 - val_loss: 3.3623 - val_accuracy: 0.3636
Epoch 7/100
10/10 [==============================] - 55s 6s/step - loss: 2.8161 - accuracy: 0.6364 - val_loss: 3.1856 - val_accuracy: 0.3939
Epoch 8/100
1

In [13]:
# Save our Model
model.save_pretrained("models/")
tokenizer.save_pretrained("tokenizers/")


('tokenizers/tokenizer_config.json',
 'tokenizers/special_tokens_map.json',
 'tokenizers/vocab.txt',
 'tokenizers/added_tokens.json')